# Chapter 16.3: Semantic compositionality

Loading libraries:

In [ ]:
using JudiLing, DataFrames, Statistics, Plots
default(fmt=:jpg)

## FRACSS

Define a small toy dataset:

In [ ]:
re_data = DataFrame("simple"=>["do", "visit", "sing", "open"],
                        "affixed"=>["redo", "revisit", "resing", "reopen"])

Get semantic vectors for both the simple and affixed vectors. For demonstration, reduce them to 5 dimensions:

In [ ]:
re_data, S = JudiLing.load_S_matrix_from_fasttext(re_data, :en, target_col="simple")
S = S[:,1:5]

In [ ]:
sample_words, A = JudiLing.load_S_matrix_from_fasttext(re_data, :en, target_col="affixed")
A = A[:,1:5]

Compute a mapping matrix from simple to affixed vectors:

In [ ]:
FRACSS = JudiLing.make_transform_matrix(S, A)

Predict the affixed vectors:

In [ ]:
Ahat = S * FRACSS

Compute the correlation matrix between the predicted and the target affixed vectors:

In [ ]:
acc_affixed, cor_affixed = JudiLing.eval_SC(Ahat, A, R=true)

Plot as heatmap:

In [ ]:
p=heatmap(cor_affixed,  xticks=(1:1:4, ["re*do", "re*visit", "re*sing", "re*open"]),
yticks=(1:1:4, re_data.affixed), xlab="Predicted semantics", ylab="Target semantics", size=(600,400),clim=(-1,1))
for row in 1:4
    for col in 1:4 
        annotate!(row,col,text.(round.(cor_affixed[row,col], digits=2), :lightblue))
    end
end
p

In [ ]:
#savefig("../fig/fracss.pdf")

## CAOSS

Define a toy dataset with a few compounds:

In [ ]:
compounds = DataFrame("word"=>["schoolwork", "homework", "housework", "schoolbus", "housekeeper"],
                    "constituent1"=>["school", "home", "house", "school", "house"],
                    "constituent2"=>["work", "work", "work", "bus", "keeper"])

Load semantic vectors for the compounds and each of the constituents. For demonstration purposes again limit the semantic vectors to 5 dimensions:

In [ ]:
compounds, T = JudiLing.load_S_matrix_from_fasttext(compounds, :en, target_col="word")
T = T[:,1:5]

In [ ]:
compounds, S_constituents1 = JudiLing.load_S_matrix_from_fasttext(compounds, :en, target_col="constituent1")
S_constituents1 = S_constituents1[:,1:5]

In [ ]:
compounds, S_constituents2 = JudiLing.load_S_matrix_from_fasttext(compounds, :en, target_col="constituent2")
S_constituents2 = S_constituents2[:,1:5]

Concatenate the semantic vectors of the constituents:

In [ ]:
O = hcat(S_constituents1, S_constituents2)

First constituent

In [ ]:
O[:,1:5]

Second consituent

In [ ]:
O[:,6:end]

Compute mapping matrix:

In [ ]:
N = JudiLing.make_transform_matrix(O, T)

To mirror how the mapping is displayed in Marelli et al. (2017), split the mapping matrix back into two matrices, one for each of the constituents:

In [ ]:
M = N[1:5, :]
H = N[6:end, :]

Reconstructing "schoolwork" step by step, first using the two individual matrices:

In [ ]:
school = S_constituents1[compounds.constituent1 .== "school",:][1:1,:]
work = S_constituents2[compounds.constituent2 .== "work", :][1:1,:]

school_const = school * M
work_const = work * H

schoolwork = school_const .+ work_const

Now show that we get the same result if we use one mapping matrix:

In [ ]:
schoolwork_direct = hcat(school, work) * N

In [ ]:
all(schoolwork .≈ schoolwork_direct)

How correlated is the predicted schoolwork vector with the original one?

In [ ]:
schoolwork_target = T[(compounds.word .== "schoolwork"),:]
cor(schoolwork, schoolwork_target, dims=2)

Estimating all vectors at once:

In [ ]:
That = O * N

Compute correlation matrix between predicted and target compound vectors:

In [ ]:
acc, cor_m = JudiLing.eval_SC(That, T, R=true)

Generate heatmap

In [ ]:
p=heatmap(cor_m,  xticks=(1:1:5, ["school*work", "home*work", "house*work", "school*bus", "house*keeper"]),
yticks=(1:1:5, compounds.word), xlab="Predicted semantics", ylab="Target semantics", size=(600,400),clim=(-1,1))
for row in 1:5
    for col in 1:5 
        annotate!(row,col,text.(round.(cor_m[row,col], digits=2), :lightblue))
    end
end
p

In [ ]:
#savefig("../fig/caoss.pdf")

# CosClassAvg

Define a toy dataset with a few words from the semantic class of "fruit":

In [ ]:
fruit = DataFrame("singular" => ["banana", "apple", "pineapple", "pear"],
                "plural" => ["bananas", "apples", "pineapples", "pears"])

Get semantic vectors for the singular and plurals from fasttext:

In [ ]:
dat, S_singulars = JudiLing.load_S_matrix_from_fasttext(fruit, :en, target_col="singular")
S_singulars = S_singulars[:, 1:5]

In [ ]:
dat, S_plurals = JudiLing.load_S_matrix_from_fasttext(fruit, :en, target_col="plural")
S_plurals = S_plurals[:, 1:5]

Compute the shift vectors of each word:

In [ ]:
S_shift = S_plurals .- S_singulars

Average over them to create the shift vector for fruit:

In [ ]:
shift_fruit = mean(S_shift, dims=1)

Predict plural semantics using the shift vector:

In [ ]:
S_plurals_predicted = S_singulars .+ shift_fruit

Compute correlation between predicted and target plural vectors:

In [ ]:
acc, cor_m_cosclassavg = JudiLing.eval_SC(S_plurals_predicted, S_plurals, R=true)

In [ ]:
p=heatmap(cor_m_cosclassavg,  xticks=(1:1:5, ["banana+s", "apple+s", "pineapple+s", "pear+s"]),
yticks=(1:1:4, dat.plural), xlab="Predicted semantics", ylab="Target semantics", size=(600,400),clim=(-1,1))
for row in 1:4
    for col in 1:4 
        annotate!(row,col,text.(round.(cor_m_cosclassavg[row,col], digits=2), :darkblue))
    end
end
p

In [ ]:
#savefig("../fig/cosclassavg.pdf")